In [1]:
import cv2
import mediapipe as mp
import numpy as np
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [2]:
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [3]:
iams_data = {"actions": ["OK", "NO", "WAVING", "CLAPPING", "SALUTE"], "subfolder_length": 30, "video_length": 60, "Data_Directory": "ActionData", "Data_Subfolder": "actionData\\data"}

actions = np.array(iams_data['actions'])
subfolder_length = iams_data['subfolder_length']
video_length = iams_data['video_length']

In [4]:
print(actions)
print(subfolder_length)
video_length
data_dir = iams_data['Data_Directory']
print(data_dir)

['OK' 'NO' 'WAVING' 'CLAPPING' 'SALUTE']
30
ActionData


In [5]:
labels = {label:num for num, label in enumerate(actions)}


In [6]:
print(labels)

{'OK': 0, 'NO': 1, 'WAVING': 2, 'CLAPPING': 3, 'SALUTE': 4}


In [7]:
import os
sequences, labels_map = [], []
for action in actions:
    for folders in range(subfolder_length):
        window = []
        for frame_num in range(video_length):
            results = np.load(os.path.join(data_dir, action, str(folders), '{}.npy'.format(frame_num) )) #change this to a wholesome path later
            window.append(results)
        sequences.append(window)
        labels_map.append(labels[action])

In [8]:
np.array(sequences).shape

(150, 60, 1629)

In [9]:
np.array(labels_map).shape

(150,)

In [10]:
X = np.array(sequences)


In [11]:
X.shape

(150, 60, 1629)

In [12]:
y = to_categorical(labels_map).astype(int)

In [13]:
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0,

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [15]:
y_test.shape

(15, 5)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [17]:
# from email import encoders
# from email.mime.base import MIMEBase
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# import base64
# import os
# from google_auth_oauthlib.flow import InstalledAppFlow

# def create_email(sender, to, subject, message_text):
#     """creates email message
#     rgs:
#     sender: Email address of the sender.
#     to: Email address of the receiver.
#     subject: The subject of the email message.
#     message_text: The text of the email message.Returns:
#     An object containing a base64url encoded email object."""
#     message = MIMEText(message_text)
#     message['to'] = to
#     message['from'] = sender 
#     message['subject'] =subject
#     return {'raw': base64.urlsafe_b64encode(message.as_string().encode()).decode()}


# def send_message(service, user_id, message):
#     """Sends email message created by create_email()
    
#     Args:
#     service: Authorized Gmail API service instance.
#     user_id: User's email address. The special value "me"
#     can be used to indicate the authenticated user.
#     message: Message to be sent.
    
#     Returns:
#         Sent Message.
#     """
#     try:
#         message = (service.users().messages().send(userId=user_id, body=message).execute())
#         print('Message Id: {}'.format(message['id']))
#         return message
#     except:
#         print ('An error occurred')
        
# def send_notification(sender, to, subject, notification):
#     #Sender is the sender email, to is the receiver email, subject is the email subject, and notification is the email body message. All the text is str object.
#     SCOPES = 'https://mail.google.com/'
#     message = create_email(sender, to, subject, notification)
#     creds = None
    
#     if os.path.exists('token.pickle'):
#         with open('token.pickle', 'rb') as token:
#             creds = pickle.load(token)
#         #We use login if no valid credentials
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow =   InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
#             creds = flow.run_local_server(port=0)
#             # Save the credentials for the next run
#         with open('token.pickle', 'wb') as token:
#             pickle.dump(creds, token)
#             service = build('gmail', 'v1', credentials=creds)
#             send_message(service, sender, message)
    

In [18]:
# import seaborn as sns
# import pandas as pd
# import pickle
# from sklearn.linear_model import LinearRegression#Loading the dataset and only using the numerical variables
# mpg = sns.load_dataset('mpg').drop(['origin', 'name'], axis = 1)
# linear_model = LinearRegression()
# try:
#     #Training the model
#     linear_model.fit(mpg.drop('mpg', axis =1), mpg['mpg'])
#     send_notification('test1@gmail.com', 'test2@gmail.com', 'Notification - Success Training', 'The model has finish')
# except:
#     send_notification('test1@gmail.com', 'test2@gmail.com', 'Notification - Failed Training', 'The model encountered error')

In [19]:
!pip install twilio


In [20]:

from tensorflow.keras.layers import Dense
from sklearn.datasets import make_classification
from twilio.rest import Client
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

def send_message(text):

    account_sid = 'AC1da11dba5edecbaaae202b74aca60386'
    auth_token = '8060eb75cf5be984580b25b778786616'

    wh_num = '+447448009810'
    sender = '+14155238886'

    client = Client(account_sid, auth_token)

    message = client.messages.create(body = text, from_= 'whatsapp:+14155238886', 
                                    to ='whatsapp:+447448009810')

class WhatsappCallBack(Callback): 
    def on_train_begin(self, logs = None):
        self.losses = [] #Initializing the list of losses
        send_message("The training has started") #Sending message on whatsapp at the beginning of the training
        self.epoch = 0

    def on_epoch_begin(self, epoch, logs = None):
        pass
    

    def on_epoch_end(self, epoch, logs = None):
        self.losses.append(logs['loss'])
        accuracy = logs.get('acc')
        loss = logs.get('loss')
        val_loss = logs.get('val_loss')
        val_acc = logs.get('val_acc')
        message = "Epoch: " + str(self.epoch + 2) + "Loss: " + str(loss)[0:5] + "Val.Loss: " + str(val_loss)[0:5] + "Acc: " + str(accuracy)[0:5] + "Val.Acc: " + str(val_acc)[0:5]
        send_message(message)
        self.epoch += 1

    def on_train_end(self, logs = None):
        send_message("The training has ended") #Sending message on whatsapp at the end of the training
        send_message(self.losses)

In [21]:
cb1 = WhatsappCallBack()

In [22]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'tanh', input_shape=(60, 1629)))
model.add(LSTM(128, return_sequences = True, activation = 'tanh'))
# model.add(LSTM(128, return_sequences = True, activation = 'tanh'))
# model.add(LSTM(256, return_sequences = True, activation = 'tanh'))
# model.add(LSTM(128, return_sequences = True, activation = 'tanh'))
model.add(LSTM(64, return_sequences = True, activation = 'tanh'))
model.add(LSTM(128, return_sequences = False, activation = 'tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [23]:
# results = [0.6, 0.3, 0.7, 0.05, 0.05]
# actions[np.argmax(results)]
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 64)            433664    
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 64)            49408     
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1

In [24]:
# import tensorflow as tf
# opt = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
import tensorflow as tf
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


EPOCHS = 10
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='categorical_accuracy',
    mode='max',
    save_best_only=True)

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
# model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

# The model weights (that are considered the best) are loaded into the model.



In [25]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback, model_checkpoint_callback, cb1])
model.load_weights(checkpoint_filepath)

Epoch 1/500
5/5 [==============================] - 6s 332ms/step - loss: 1.6565 - categorical_accuracy: 0.2009
Epoch 2/500
5/5 [==============================] - 1s 175ms/step - loss: 1.5988 - categorical_accuracy: 0.3207
Epoch 3/500
5/5 [==============================] - 1s 163ms/step - loss: 1.6228 - categorical_accuracy: 0.2066
Epoch 4/500
5/5 [==============================] - 1s 157ms/step - loss: 1.6101 - categorical_accuracy: 0.1999
Epoch 5/500
5/5 [==============================] - 1s 153ms/step - loss: 1.6157 - categorical_accuracy: 0.2192
Epoch 6/500
5/5 [==============================] - 1s 165ms/step - loss: 1.6005 - categorical_accuracy: 0.2299
Epoch 7/500
5/5 [==============================] - 1s 162ms/step - loss: 1.5640 - categorical_accuracy: 0.2913
Epoch 8/500
5/5 [==============================] - 1s 171ms/step - loss: 1.5395 - categorical_accuracy: 0.4056
Epoch 9/500
5/5 [==============================] - 1s 190ms/step - loss: 1.4736 - categorical_accuracy: 0.2877
E

5/5 [==============================] - 1s 147ms/step - loss: 0.8133 - categorical_accuracy: 0.6739
Epoch 75/500
5/5 [==============================] - 1s 150ms/step - loss: 0.7381 - categorical_accuracy: 0.6917
Epoch 76/500
5/5 [==============================] - 1s 147ms/step - loss: 0.9482 - categorical_accuracy: 0.6210
Epoch 77/500
5/5 [==============================] - 1s 150ms/step - loss: 0.8550 - categorical_accuracy: 0.6463
Epoch 78/500
5/5 [==============================] - 1s 140ms/step - loss: 0.8908 - categorical_accuracy: 0.5982
Epoch 79/500
5/5 [==============================] - 1s 155ms/step - loss: 0.6565 - categorical_accuracy: 0.7634
Epoch 80/500
5/5 [==============================] - 1s 149ms/step - loss: 0.7089 - categorical_accuracy: 0.7146
Epoch 81/500
5/5 [==============================] - 1s 150ms/step - loss: 0.6615 - categorical_accuracy: 0.7337
Epoch 82/500
5/5 [==============================] - 1s 149ms/step - loss: 0.6525 - categorical_accuracy: 0.7467
Epoch

5/5 [==============================] - 1s 154ms/step - loss: 0.6800 - categorical_accuracy: 0.6949
Epoch 147/500
5/5 [==============================] - 1s 147ms/step - loss: 0.5901 - categorical_accuracy: 0.7384
Epoch 148/500
5/5 [==============================] - 1s 149ms/step - loss: 0.5748 - categorical_accuracy: 0.7847
Epoch 149/500
5/5 [==============================] - 1s 147ms/step - loss: 0.5368 - categorical_accuracy: 0.7868
Epoch 150/500
5/5 [==============================] - 1s 160ms/step - loss: 0.5038 - categorical_accuracy: 0.8243
Epoch 151/500
5/5 [==============================] - 1s 147ms/step - loss: 0.5397 - categorical_accuracy: 0.7836
Epoch 152/500
5/5 [==============================] - 1s 145ms/step - loss: 0.4691 - categorical_accuracy: 0.7985
Epoch 153/500
5/5 [==============================] - 1s 147ms/step - loss: 0.5673 - categorical_accuracy: 0.7529
Epoch 154/500
5/5 [==============================] - 1s 148ms/step - loss: 0.5074 - categorical_accuracy: 0.78

5/5 [==============================] - 1s 146ms/step - loss: 0.4250 - categorical_accuracy: 0.8276
Epoch 219/500
5/5 [==============================] - 1s 161ms/step - loss: 0.5517 - categorical_accuracy: 0.8252
Epoch 220/500
5/5 [==============================] - 1s 144ms/step - loss: 0.6550 - categorical_accuracy: 0.7772
Epoch 221/500
5/5 [==============================] - 1s 154ms/step - loss: 0.5645 - categorical_accuracy: 0.7594
Epoch 222/500
5/5 [==============================] - 1s 148ms/step - loss: 0.5465 - categorical_accuracy: 0.7793
Epoch 223/500
5/5 [==============================] - 1s 156ms/step - loss: 0.4970 - categorical_accuracy: 0.7947
Epoch 224/500
5/5 [==============================] - 1s 150ms/step - loss: 0.4763 - categorical_accuracy: 0.8001
Epoch 225/500
5/5 [==============================] - 1s 153ms/step - loss: 0.4403 - categorical_accuracy: 0.8055
Epoch 226/500
5/5 [==============================] - 1s 147ms/step - loss: 0.4136 - categorical_accuracy: 0.84

5/5 [==============================] - 1s 154ms/step - loss: 0.3002 - categorical_accuracy: 0.8753
Epoch 291/500
5/5 [==============================] - 1s 152ms/step - loss: 0.3430 - categorical_accuracy: 0.8554
Epoch 292/500
5/5 [==============================] - 1s 150ms/step - loss: 0.3061 - categorical_accuracy: 0.9189
Epoch 293/500
5/5 [==============================] - 1s 148ms/step - loss: 0.2980 - categorical_accuracy: 0.8905
Epoch 294/500
5/5 [==============================] - 1s 152ms/step - loss: 0.2725 - categorical_accuracy: 0.8966
Epoch 295/500
5/5 [==============================] - 1s 140ms/step - loss: 0.4965 - categorical_accuracy: 0.7941
Epoch 296/500
5/5 [==============================] - 1s 149ms/step - loss: 0.4294 - categorical_accuracy: 0.8412
Epoch 297/500
5/5 [==============================] - 1s 148ms/step - loss: 0.3351 - categorical_accuracy: 0.8574
Epoch 298/500
5/5 [==============================] - 1s 149ms/step - loss: 0.2723 - categorical_accuracy: 0.90

5/5 [==============================] - 1s 144ms/step - loss: 0.0408 - categorical_accuracy: 0.9919
Epoch 363/500
5/5 [==============================] - 1s 151ms/step - loss: 0.0550 - categorical_accuracy: 0.9867
Epoch 364/500
5/5 [==============================] - 1s 146ms/step - loss: 0.0396 - categorical_accuracy: 0.9919
Epoch 365/500
5/5 [==============================] - 1s 154ms/step - loss: 0.0380 - categorical_accuracy: 0.9919
Epoch 366/500
5/5 [==============================] - 1s 144ms/step - loss: 0.0315 - categorical_accuracy: 0.9945
Epoch 367/500
5/5 [==============================] - 1s 151ms/step - loss: 0.0284 - categorical_accuracy: 0.9962
Epoch 368/500
5/5 [==============================] - 1s 147ms/step - loss: 0.0608 - categorical_accuracy: 0.9812
Epoch 369/500
5/5 [==============================] - 1s 150ms/step - loss: 0.2707 - categorical_accuracy: 0.9128
Epoch 370/500
5/5 [==============================] - 1s 147ms/step - loss: 0.0908 - categorical_accuracy: 0.97

5/5 [==============================] - 1s 151ms/step - loss: 0.0192 - categorical_accuracy: 0.9975
Epoch 435/500
5/5 [==============================] - 1s 144ms/step - loss: 0.0463 - categorical_accuracy: 0.9867
Epoch 436/500
5/5 [==============================] - 1s 155ms/step - loss: 0.0322 - categorical_accuracy: 0.9919
Epoch 437/500
5/5 [==============================] - 1s 148ms/step - loss: 0.0336 - categorical_accuracy: 0.9919
Epoch 438/500
5/5 [==============================] - 1s 152ms/step - loss: 0.0223 - categorical_accuracy: 0.9962
Epoch 439/500
5/5 [==============================] - 1s 153ms/step - loss: 0.0234 - categorical_accuracy: 0.9962
Epoch 440/500
5/5 [==============================] - 1s 155ms/step - loss: 0.0241 - categorical_accuracy: 0.9945
Epoch 441/500
5/5 [==============================] - 1s 151ms/step - loss: 0.0228 - categorical_accuracy: 0.9945
Epoch 442/500
5/5 [==============================] - 1s 164ms/step - loss: 0.0207 - categorical_accuracy: 0.99

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 64)            433664    
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 64)            49408     
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1

In [27]:
results= model.predict(X_test)

In [28]:
actions[np.argmax(results[2])]

'SALUTE'

In [29]:
actions[np.argmax(results[2])]

'SALUTE'

In [30]:
model.save('IAMSaction_1.h5')

In [31]:
model.load_weights('IAMSaction_1.h5')

In [32]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:
yhat = model.predict(X_test)

In [34]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
ytrue

[3, 2, 4, 3, 3, 1, 0, 0, 2, 1, 2, 0, 2, 2, 3]

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[12,  0],
        [ 0,  3]],

       [[13,  0],
        [ 0,  2]],

       [[10,  0],
        [ 0,  5]],

       [[11,  0],
        [ 0,  4]],

       [[14,  0],
        [ 0,  1]]])

In [36]:
accuracy_score(ytrue, yhat)

1.0

In [43]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    output_frame = np.vectorize(output_frame)
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [44]:
image = cv2.imread('imgs/clap_1.jpeg')
# cv2.imshow('Output', image)
plt.figure(figsize=(18,18))

image.shape
plt.imshow(prob_viz(results, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1296x1296 with 0 Axes>